In [1]:
import pandas as pd
import numpy as np
import sympy
from sympy.solvers.inequalities import reduce_rational_inequalities,reduce_inequalities
from itertools import chain
from collections import defaultdict
from IPython.display import display

from sympy.parsing.sympy_parser import parse_expr
from sympy.parsing.sympy_parser import standard_transformations
from sympy.printing.mathml import print_mathml

from IPython.display import display, Markdown, Latex
import os

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def make_symbols_positive(sym):
    return sympy.symbols("{}".format(sym),positive =True)

def make_symbols_function(sym):
    return sympy.symbols("{}".format(sym),cl)

def subs_function(target,list_):
    return sympy.simplify(target.subs(list_))

# Data_loading

In [3]:
file_list = os.listdir(r"C:\Users\Samsung\Desktop\Writing\Graduation\Solve\version_1")
file_list_xlsx = [f for f in file_list if f.endswith(".xlsx")]
#file_FB = [f for f in file_list_xlsx if f.contains("FB")]
#file_WH = [f for f in file_list_xlsx if f.contains("WH")]
#file_QD = [f for f in file_list_xlsx if f.contains("QD")]

In [4]:
file_name = [li.replace(".xlsx","") for li in file_list_xlsx]
file_name

['BDD_FB',
 'BDD_QD',
 'BDD_WH',
 'BDS_FB',
 'BDS_QD',
 'BDS_WH',
 'DD_FB',
 'DD_QD',
 'DD_WH',
 'ND_FB',
 'ND_QD',
 'ND_WH',
 'SD_FB',
 'SD_QD',
 'SD_WH']

In [5]:
for fn in file_name:
    path_ = r"C:\Users\Samsung\Desktop\Writing\Graduation\Solve\version_1\{}.xlsx"
    globals() ["{}".format(fn)] = pd.read_excel(path_.format(fn),index_col=0,header=None)
    try:
        globals() ["{}".format(fn)] = globals() ["{}".format(fn)].rename(index={"profit_all":"pro_all"})
    except:
        pass

In [6]:
alpha, beta, p, c, w, theta, Delta, delta_s, delta_d, lambda_ = make_symbols_positive("alpha, beta, p, c, w, theta , Delta, delta_s, delta_d, lambda")

In [7]:
parse_dic = {"alpha":alpha, "beta":beta, "p":p, "c":c,"w":w, "theta":theta,"Delta": Delta, "delta_s":delta_s, "delta_d":delta_d,"lambda_": lambda_ }

In [8]:
for fn in file_name:
    globals() ["{}".format(fn)][1] = globals() ["{}".format(fn)][1].map(lambda x: parse_expr(x.replace("^","**").replace("lambda","lambda_"),parse_dic))

# Data combined by Case_

In [9]:
case_  = ["ND","DD","SD","BDD","BDS"]

In [10]:
for ca in case_:
    FB_ = globals ()["{}_FB".format(ca)].rename(columns={1:"FB"})
    WH_ = globals ()["{}_WH".format(ca)].rename(columns={1:"WH"})
    QD_ = globals ()["{}_QD".format(ca)].rename(columns={1:"QD"})
    globals ()["{}_all".format(ca)] = pd.concat([FB_,WH_,QD_],join="outer",axis=1)

In [11]:
DD_all.head()

,FB,WH,QD
p,(alpha + beta*c)/(2*beta),(3*alpha*delta_d + beta*c - beta*delta_d*lambd...,(3*alpha*delta_d + 2*beta*c - beta*delta_d*lam...
q,alpha/2 - beta*c/2,(alpha*delta_d - beta*c + beta*delta_d*lambda ...,(alpha*delta_d - 2*beta*c + beta*delta_d*lambd...
D,delta_d*(alpha/2 - beta*c/2),alpha*delta_d/4 - beta*c/4 + beta*delta_d*lamb...,alpha*delta_d/4 - beta*c/2 + beta*delta_d*lamb...
pro_all,delta_d*(alpha - beta*c)**2/(4*beta),3*(-alpha*delta_d + beta*c - beta*delta_d*lamb...,(alpha*delta_d + beta*delta_d*lambda - beta*la...
h1,-2*beta*delta_d,NaN,NaN


In [12]:
ND_all

,FB,WH,QD
p,(alpha + beta*c)/(2*beta),(3*alpha + beta*c)/(4*beta),(3*alpha + 2*beta*c - beta*w)/(4*beta)
q,alpha/2 - beta*c/2,alpha/4 - beta*c/4,alpha/4 - beta*c/2 + beta*w/4
D,alpha/2 - beta*c/2,alpha/4 - beta*c/4,alpha/4 - beta*c/2 + beta*w/4
pro_all,(alpha - beta*c)**2/(4*beta),3*(alpha - beta*c)**2/(16*beta),(3*alpha**2 - 8*alpha*beta*c + 2*alpha*beta*w ...
h1,-2*beta,NaN,NaN
w,NaN,(alpha + beta*c)/(2*beta),w
p_w,NaN,(alpha + beta*w)/(2*beta),NaN
S,NaN,alpha/4 - beta*c/4,alpha/4 - beta*c/2 + beta*w/4
T,NaN,(alpha + beta*c)/(2*beta),(alpha + 2*beta*c + beta*w)/(4*beta)
pro_r,NaN,(alpha - beta*c)**2/(16*beta),(alpha - beta*w)*(alpha/4 - beta*c/2 + beta*w/...


# ND

In [22]:
for key,value in ND_all["FB"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


(alpha + beta*c)/(2*beta)


q


-(-alpha + beta*c)/2


D


...

nan


h1_s_th


nan


h2_s


nan

In [23]:
for key,value in ND_all["WH"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


(3*alpha + beta*c)/(4*beta)


q


-(-alpha + beta*c)/4


D


...

nan


h1_s_th


nan


h2_s


nan

In [25]:
for key,value in ND_all["QD"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


-(-3*alpha - 2*beta*c + beta*w)/(4*beta)


q


(alpha - 2*beta*c + beta*w)/4


D


...

-(-2*alpha*beta*theta + alpha + beta*w)/(2*beta*(beta*theta - 1))


h1_s_th


beta*(-alpha + beta*w)*(alpha*beta*theta + 2*alpha - 2*beta**2*c*theta + beta**2*theta*w + 2*beta*c - 4*beta*w)/(2*(beta*theta - 1)**4)


h2_s


-beta**4*(-alpha*theta + beta*c*theta - c + w)**2/(4*(beta*theta - 1)**6)

# DD

In [26]:
for key,value in DD_all["FB"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


(alpha + beta*c)/(2*beta)


q


-(-alpha + beta*c)/2


D


...

nan


h1_s_th


nan


h2_s


nan

In [27]:
for key,value in DD_all["WH"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


(3*alpha*delta_d + beta*c - beta*delta_d*lambda + beta*lambda)/(4*beta*delta_d)


q


-(-alpha*delta_d + beta*c - beta*delta_d*lambda + beta*lambda)/(4*delta_d)


D


...

nan


h1_s_th


nan


h2_s


nan

In [28]:
for key,value in DD_all["QD"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


-(-3*alpha*delta_d - 2*beta*c + beta*delta_d*lambda - beta*lambda + beta*w)/(4*beta*delta_d)


q


(alpha*delta_d - 2*beta*c + beta*delta_d*lambda - beta*lambda + beta*w)/(4*delta_d)


D


...

-(-2*alpha*beta*theta + alpha*delta_d - beta*delta_d*lambda + beta*lambda + beta*w)/(2*beta*(beta*theta - delta_d))


h1_s_th


beta*(-alpha*delta_d - beta*delta_d*lambda + beta*lambda + beta*w)*(alpha*beta*delta_d*theta + 2*alpha*delta_d**2 - 2*beta**2*c*theta + beta**2*delta_d*lambda*theta - beta**2*lambda*theta + beta**2*theta*w + 2*beta*c*delta_d + 2*beta*delta_d**2*lambda - 2*beta*delta_d*lambda - 4*beta*delta_d*w)/(2*(beta*theta - delta_d)**4)


h2_s


-beta**4*(-alpha*delta_d*theta + beta*c*theta - beta*delta_d*lambda*theta + beta*lambda*theta - c*delta_d + delta_d*w)**2/(4*(beta*theta - delta_d)**6)

# SD

In [29]:
for key,value in SD_all["FB"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


(alpha + beta*c)/(2*beta)


q


-(-alpha + beta*c)/2


D


...

nan


h1_s_th


nan


h2_s


nan

In [30]:
for key,value in SD_all["WH"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


(3*alpha + beta*c)/(4*beta)


q


-(-alpha + beta*c)/4


D


...

nan


h1_s_th


nan


h2_s


nan

In [31]:
for key,value in SD_all["QD"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


-(-3*alpha - 2*beta*c + beta*w)/(4*beta)


q


(alpha - 2*beta*c + beta*w)/4


D


...

-(-2*alpha*beta*delta_s*theta + alpha + beta*w)/(2*beta*(beta*delta_s*theta - 1))


h1_s_th


beta*delta_s**3*(-alpha + beta*w)*(alpha*beta*delta_s*theta + 2*alpha - 2*beta**2*c*delta_s*theta + beta**2*delta_s*theta*w + 2*beta*c - 4*beta*w)/(2*(beta*delta_s*theta - 1)**4)


h2_s


-beta**4*delta_s**4*(-alpha*delta_s*theta + beta*c*delta_s*theta - c + w)**2/(4*(beta*delta_s*theta - 1)**6)

# BDD

In [13]:
for key,value in BDD_all["FB"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


(alpha + beta*c)/(2*beta)


q


-(-alpha + beta*c)/2


D


...

nan


h1_s_th


nan


h2_s


nan

In [14]:
for key,value in BDD_all["WH"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


(3*alpha*delta_d + beta*c*delta_s - beta*delta_d*lambda + beta*delta_s*lambda)/(4*beta*delta_d)


q


-(-alpha*delta_d + beta*c*delta_s - beta*delta_d*lambda + beta*delta_s*lambda)/(4*delta_d)


D


...

nan


h1_s_th


nan


h2_s


nan

In [15]:
for key,value in BDD_all["QD"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


-(-3*alpha*delta_d - 2*beta*c*delta_s + beta*delta_d*lambda - beta*delta_s*lambda + beta*delta_s*w)/(4*beta*delta_d)


q


(alpha*delta_d - 2*beta*c*delta_s + beta*delta_d*lambda - beta*delta_s*lambda + beta*delta_s*w)/(4*delta_d)


D


...

-(-2*alpha*beta*delta_s**2*theta + alpha*delta_d - beta*delta_d*lambda + beta*delta_s*lambda + beta*delta_s*w)/(2*beta*(beta*delta_s**2*theta - delta_d))


h1_s_th


beta*delta_s**4*(-alpha*delta_d - beta*delta_d*lambda + beta*delta_s*lambda + beta*delta_s*w)*(alpha*beta*delta_d*delta_s**2*theta + 2*alpha*delta_d**2 - 2*beta**2*c*delta_s**3*theta + beta**2*delta_d*delta_s**2*lambda*theta - beta**2*delta_s**3*lambda*theta + beta**2*delta_s**3*theta*w + 2*beta*c*delta_d*delta_s + 2*beta*delta_d**2*lambda - 2*beta*delta_d*delta_s*lambda - 4*beta*delta_d*delta_s*w)/(2*(beta*delta_s**2*theta - delta_d)**4)


h2_s


-beta**4*delta_s**8*(-alpha*delta_d*delta_s*theta + beta*c*delta_s**2*theta - beta*delta_d*delta_s*lambda*theta + beta*delta_s**2*lambda*theta - c*delta_d + delta_d*w)**2/(4*(beta*delta_s**2*theta - delta_d)**6)

# BDS

In [16]:
for key,value in BDS_all["FB"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


(alpha + beta*c)/(2*beta)


q


-(-alpha + beta*c)/2


D


...

nan


h1_s_th


nan


h2_s


nan

In [17]:
for key,value in BDS_all["WH"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


(3*alpha + beta*c)/(4*beta)


q


-(-alpha + beta*c)/4


D


...

nan


h1_s_th


nan


h2_s


nan

In [18]:
for key,value in BDS_all["QD"].items():
    print(key)
    try:
        display(value.factor())
    except:
        display(value)
    print()

p


-(-3*alpha - 2*beta*c + beta*w)/(4*beta)


q


(alpha - 2*beta*c + beta*w)/4


D


...

-(-2*alpha*beta*delta_s*theta + alpha + beta*w)/(2*beta*(beta*delta_s*theta - 1))


h1_s_th


beta*delta_s**3*(-alpha + beta*w)*(alpha*beta*delta_s*theta + 2*alpha - 2*beta**2*c*delta_s*theta + beta**2*delta_s*theta*w + 2*beta*c - 4*beta*w)/(2*(beta*delta_s*theta - 1)**4)


h2_s


-beta**4*delta_s**4*(-alpha*delta_s*theta + beta*c*delta_s*theta - c + w)**2/(4*(beta*delta_s*theta - 1)**6)

# data Concat

In [12]:
# concat 함수
def concat_all(target,case_):
    all_ = pd.DataFrame()
    for ca in case_:
        try:
            profit_all = pd.DataFrame(globals ()["{}_all".format(ca)].loc[target]).rename(columns={target:ca})
            all_ = pd.concat([all_,profit_all],axis=1)
        except:
            pass
    return all_

In [14]:
concat_all("pro_all",case_)

,ND,DD,SD,BDD,BDS
FB,(alpha - beta*c)**2/(4*beta),delta_d*(alpha - beta*c)**2/(4*beta),delta_s*(alpha - beta*c)**2/(4*beta),delta_d*(alpha - beta*c)**2/(4*beta),delta_s*(alpha - beta*c)**2/(4*beta)
WH,3*(alpha - beta*c)**2/(16*beta),3*(-alpha*delta_d + beta*c - beta*delta_d*lamb...,3*delta_s*(alpha - beta*c)**2/(16*beta),3*(alpha*delta_d - beta*c*delta_s + beta*delta...,3*delta_s*(alpha - beta*c)**2/(16*beta)
QD,(3*alpha**2 - 8*alpha*beta*c + 2*alpha*beta*w ...,(alpha*delta_d + beta*delta_d*lambda - beta*la...,delta_s*(3*alpha**2 - 8*alpha*beta*c + 2*alpha...,(alpha*delta_d + beta*delta_d*lambda - beta*de...,delta_s*(3*alpha**2 - 8*alpha*beta*c + 2*alpha...


In [23]:
BDD_all

,FB,WH,QD
p,(alpha + beta*c)/(2*beta),(3*alpha*delta_d + beta*c*delta_s - beta*delta...,(3*alpha*delta_d + 2*beta*c*delta_s - beta*del...
q,alpha/2 - beta*c/2,(alpha*delta_d - beta*c*delta_s + beta*delta_d...,(alpha*delta_d - 2*beta*c*delta_s + beta*delta...
D,delta_d*(alpha/2 - beta*c/2),alpha*delta_d/4 - beta*c*delta_s/4 + beta*delt...,alpha*delta_d/4 - beta*c*delta_s/2 + beta*delt...
pro_all,delta_d*(alpha - beta*c)**2/(4*beta),3*(alpha*delta_d - beta*c*delta_s + beta*delta...,(alpha*delta_d + beta*delta_d*lambda - beta*de...
h1,-2*beta*delta_d,NaN,NaN
w,NaN,(alpha*delta_d + beta*c*delta_s + beta*delta_d...,w
p_w,NaN,(alpha*delta_d + beta*delta_s*w - lambda*(beta...,NaN
S,NaN,delta_s*(alpha*delta_d - beta*c*delta_s + beta...,delta_s*(alpha*delta_d - 2*beta*c*delta_s + be...
T,NaN,(alpha*delta_d + beta*c*delta_s + beta*delta_d...,(alpha*delta_d + 2*beta*c*delta_s + beta*delta...
pro_r,NaN,(alpha*delta_d - beta*c*delta_s + beta*delta_d...,(alpha*delta_d + beta*delta_d*lambda - beta*de...


In [19]:
DD_all["WH"]["h1_r_p"]

-2*beta*delta_d

In [22]:
DD_all["WH"]["h1_s_w"]

-beta/delta_d

In [17]:
(DD_all["WH"]["pro_all"] - BDD_all["WH"]["pro_all"]).factor()

-3*(c + lambda)*(delta_s - 1)*(-2*alpha*delta_d + beta*c*delta_s + beta*c - 2*beta*delta_d*lambda + beta*delta_s*lambda + beta*lambda)/(16*delta_d)